In [48]:
import traceback

import requests
from bs4 import BeautifulSoup

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import folium

In [13]:
# Send a GET request to the web page
url = "https://www.bmkg.go.id/gempabumi-dirasakan.html"
response = requests.get(url)

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the desired elements on the page
table = soup.find('table', {'class': 'table table-hover table-striped'})

# Extract the table headers
headers = [header.get_text(strip=True) for header in table.find_all('th')]

# Extract the table rows
rows = table.find_all('tr')
data = []
for row in rows:
    columns = row.find_all('td')
    row_data = [column.get_text(strip=True) for column in columns]
    data.append(row_data)

# Create a pandas DataFrame
df = pd.DataFrame(data, columns=headers).iloc[1:,1:]

# Convert the "waktu gempa" column to datetime format
df["Waktu Gempa"] = pd.to_datetime(df["Waktu Gempa"], format="%d/%m/%Y%H:%M:%S WIB")

# Convert 'magnitudo' column to float
df['Magnitudo'] = pd.to_numeric(df['Magnitudo'], errors='coerce')

# Extract numerical part from "Kedalaman" column and convert to float
df['Kedalaman'] = df['Kedalaman'].str.extract(r'(\d+)', expand=False).astype(float)

# Split the "Lintang - Bujur" column into four columns
df[['Lintang_Value', 'Lintang_String', 'Bujur_Value', 'Bujur_String']] = df['Lintang - Bujur'].str.split(expand=True)

# Create a new column "Latitude" based on Lintang_Value and Lintang_String
df['Latitude'] = df['Lintang_Value'].astype(float)
df.loc[df['Lintang_String'] == 'LS', 'Latitude'] *= -1

# Create a new column "Longitude" based on Bujur_Value and Bujur_String
df['Longitude'] = df['Bujur_Value'].astype(float)
df.loc[df['Bujur_String'] == 'BB', 'Longitude'] *= -1

# Drop the original "Lintang - Bujur" column
df.drop(['Lintang - Bujur', 'Lintang_Value', 'Lintang_String', 'Bujur_Value', 'Bujur_String', 'Dirasakan (Skala MMI)'], axis=1, inplace=True)

# Create a GeoDataFrame with Longitude and Latitude as the geometry
geometry = gpd.points_from_xy(df['Longitude'], df['Latitude'])
gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf = gdf.rename(columns = {_:_.lower().replace(" ","_") for _ in gdf.columns})
gdf.drop(['longitude', 'latitude'], axis=1, inplace=True)

gdf.head(3)

,waktu_gempa,magnitudo,kedalaman,geometry
1,2023-06-05 23:13:24,4.4,11.0,POINT (118.28000 0.82000)
2,2023-06-05 11:57:47,3.3,10.0,POINT (120.74000 1.23000)
3,2023-06-04 20:48:39,4.6,10.0,POINT (107.89000 -8.43000)


In [89]:
# Create a Folium map centered at the mean of the points' coordinates
center = [gdf.geometry.y.mean(), gdf.geometry.x.mean()]
m = folium.Map(location=center, zoom_start=5)

# Create a feature group to hold the circle markers
marker_group = folium.FeatureGroup(name='Markers')

# Iterate over each row in the GeoDataFrame
for idx, row in gdf.iterrows():
    # Get the magnitudo and kedalaman values
    magnitudo = row['magnitudo']
    kedalaman = row['kedalaman']
    waktu_gempa = row['waktu_gempa']
    
    # Get the point's coordinates
    lon, lat = row.geometry.x, row.geometry.y
    
    # Create a CircleMarker for each point with size based on magnitudo and color based on kedalaman
    circle = folium.CircleMarker(
        location=(lat, lon), 
        radius=magnitudo*5,
        color=kedalaman,  # this is the color of the border
        weight=1,  # thickness of the border
        opacity=0.3,  # this is the alpha for the border
        fill_color='red',  # fill is inside the circle
        fill_opacity=0.1,  # we will make that less opaque so we can see layers
        )
    
    # Create HTML content for the popup
    popup_content = f"Time: {waktu_gempa}<br>Magnitudo: {magnitudo}"
    
    # Add the popup to the CircleMarker
    circle.add_child(folium.Popup(popup_content))
    
    # Create a JavaScript callback function to display the popup when clicked
    callback = f"""
        function showPopup(e) {{
            var popup = L.popup()
                .setLatLng(e.latlng)
                .setContent('{popup_content}')
                .openOn(mymap);
        }}
        circle.on('click', showPopup);
    """
    
    # Add the JavaScript callback function to the circle marker's options
    circle.add_to(marker_group)

# Add the marker group to the map
marker_group.add_to(m)


m.save('earthquakes.html')

In [94]:
m